In [0]:
dbutils.widgets.text("directory", "/Volumes/melissap/ardent_demo_bcbs_bronze/mrfs/in_network")
dbutils.widgets.text("output_folder", "/Volumes/mgiglia/dev_matthew_giglia_price_transparency/landing/in-network/uncompressed")

In [0]:
directory_use = dbutils.widgets.get("directory")
output_folder_use = dbutils.widgets.get("output_folder")

In [0]:
print(f"""
      directory_use: {directory_use}
      output_folder_use: {output_folder_use}
""")

In [0]:
import os
import gzip
import shutil
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql.functions import pandas_udf, lit
import pandas as pd

In [0]:
# source_path = "/Volumes/melissap/ardent_demo_bcbs_bronze/mrfs/in_network/2025-08_020_02E0_in-network-rates_2_of_2.json.gz?&Expires=1758376850&Signature=WnmraxwcwbZSbRQMY7aiZWbaAlCjKkF7R8tvTQHrASzbCCBFWcojQHcZpdGNaUY~dtTECuZYEJbNyBnItPnDhjR0w53R~aHIMa5MSkHNl66msesHo-ijchGHgdTEgkNxfoRPL4QTRw7iKlDE7rASmgFc9f8ZggxFNPc6-Mz1eWIrX2WoVaQI7tWWuV4FjJliGD9KxIitgiKHUZ09lATWjoKLGm25MdYiNEk01GVASymiMxG4DwWXgQvfi3wdqeSJCsk0QrO5wF4V-5hPrsIa3P1gHUg2NS1gaE4AzzH~WVMDnX-DwztvALLw4FE4I010RpSWyZ67lPX-pXEmuqLK9Q__&Key-Pair-Id=K27TQMT39R1C8A"
# output_folder = "/Volumes/mgiglia/dev_matthew_giglia_price_transparency/landing/in-network/uncompressed"

In [0]:
def decompress_gzip_file(source_path, output_folder):
    """
    Decompresses a gzipped file to a specified output folder, or copies the file if not compressed.
    Cleans the filename by removing any characters to the right of .json.gz or .json.gzip.
    Args:
        source_path (str): Path to the gzipped file in Unity Catalog volume
        output_folder (str): Destination folder for decompressed file
    Returns:
        tuple: (success_status, output_path_or_error_message)
    """
    try:
        # Validate input parameters
        if not source_path or not output_folder:
            return False, "Error: Source path or output folder is empty"
        # Check if source file exists
        if not os.path.exists(source_path):
            return False, f"Error: Source file does not exist: {source_path}"
        # Clean the filename by removing query params and anything after .json.gz or .json.gzip
        base_filename = os.path.basename(source_path)
        output_filename = base_filename.lower().split("?")[0]
        if output_filename.endswith(('.gz', '.gzip')):
            output_filename = output_filename.split(".json.gz")[0] + ".json"
        elif output_filename.endswith('.json'):
            pass
        else:
            raise Exception("Error: The file is not a JSON file")
            # # Remove query params if present
            # output_filename = base_filename.split("?")[0]
        print(f""" 
              base_filename: {base_filename}
              output_filename: {output_filename}
        """)
        output_path = os.path.join(output_folder, output_filename)
        # Create output directory if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)
        # If file is compressed, decompress it
        if ".gz" in base_filename.lower() or ".gzip" in base_filename.lower():
        # if base_filename.lower().endswith(('.gz', '.gzip')):
            try:
                with gzip.open(source_path, 'rb') as gz_file:
                    with open(output_path, 'wb') as output_file:
                        shutil.copyfileobj(gz_file, output_file, length=1024*1024)  # 1MB chunks
                # Verify the decompressed file was created and has content
                if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
                    return True, output_path
                else:
                    return False, "Error: Decompressed file is empty or was not created"
            except gzip.BadGzipFile:
                return False, f"Error: File is not a valid gzip file: {source_path}"
            except OSError as e:
                return False, f"Error: Failed to read gzip file - {str(e)}"
            except Exception as e:
                return False, f"Error during decompression: {str(e)}"
        else:
            # If file is not compressed, just copy it to the output folder
            try:
                shutil.copy2(source_path, output_path)
                if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
                    return True, output_path
                else:
                    return False, "Error: Copied file is empty or was not created"
            except Exception as e:
                return False, f"Error during file copy: {str(e)}"
    except PermissionError:
        return False, f"Error: Permission denied accessing {source_path} or {output_folder}"
    except Exception as e:
        return False, f"Error: Unexpected error - {str(e)}"

In [0]:
# decompress_gzip_file(source_path, output_folder)

In [0]:
# List all files (not directories) in the directory
file_list = [os.path.join(directory_use, f) for f in os.listdir(directory_use) if os.path.isfile(os.path.join(directory_use, f))]

# Create Spark DataFrame
df_files = spark.createDataFrame([(f,) for f in file_list], ["file_path"])

# display(df_files)

In [0]:
# Define the return schema for the UDF
decompress_result_schema = StructType([
    StructField("success", BooleanType(), True),
    StructField("message", StringType(), True)
])

In [0]:
@pandas_udf(returnType=decompress_result_schema)
def decompress_gzip_arrow_udf(source_paths: pd.Series, output_folders: pd.Series) -> pd.DataFrame:
    """
    Arrow-optimized pandas UDF for batch processing of gzip decompression
    Args:
        source_paths: Pandas Series of source file paths
        output_folders: Pandas Series of output folder paths
    Returns:
        DataFrame: Contains success and message columns
    """
    results = []
    for source_path, output_folder in zip(source_paths, output_folders):
        success, message = decompress_gzip_file(source_path, output_folder)
        results.append({'success': success, 'message': message})
    df = pd.DataFrame(results)
    # Ensure correct dtypes and non-nullable columns
    df['success'] = df['success'].astype(bool).fillna(False)
    df['message'] = df['message'].astype(str).fillna('')
    return df

In [0]:
def apply_gzip_decompression_arrow(df, source_column, output_folder_path):
    """
    Apply the Arrow-optimized decompression UDF to a DataFrame
    """
    result_df = df.withColumn(
        "decompress_result",
        decompress_gzip_arrow_udf(
            col(source_column),
            lit(output_folder_path)
        )
    )
    # # Extract success and message from struct
    # final_df = result_df.select(
    #     "*",
    #     col("decompress_result.success").alias("decompress_success"),
    #     col("decompress_result.message").alias("decompress_message")
    # ).drop("decompress_result")
    # return final_df
    return result_df

In [0]:
result_df = apply_gzip_decompression_arrow(
  df = df_files
  ,source_column = "file_path"
  ,output_folder_path=output_folder
)

In [0]:
result_df.write.mode("append").saveAsTable("mgiglia.dev_matthew_giglia_price_transparency.file_copy_results")

In [0]:
# directories_and_output_folders = [
#     {
#         "directory": "/Volumes/melissap/ardent_demo_bcbs_bronze/mrfs/in_network",
#         "output_folder": "/Volumes/mgiglia/dev_matthew_giglia_price_transparency/landing/in-network/uncompressed"
#     },
#     {
#         "directory": "/Volumes/melissap/ardent_demo_bcbs_bronze/mrfs/allowed_amount",
#         "output_folder": "/Volumes/mgiglia/dev_matthew_giglia_price_transparency/landing/allowed-amount/uncompressed"
#     }
# ]

In [0]:
# directories_and_output_folders